In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.8 MB/s eta 0:00:00


In [4]:
!pip install sentencepiece -qq
!pip install tokenizer -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.9 MB/s eta 0:00:00


In [6]:
# Build class CNN

import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):

        super().__init__()

        self.fc_input = nn.Linear(embedding_dim,embedding_dim)

        self.conv_0 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[0])

        self.conv_1 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[1])

        self.conv_2 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[2])

        self.conv_3 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[3])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, encoded):

        #embedded = [batch size, sent len, emb dim]
        embedded = self.fc_input(encoded)
        #print(embedded.shape)

        embedded = embedded.permute(0, 2, 1)
        #print(embedded.shape)

        #embedded = [batch size, emb dim, sent len]

        conved_0 = F.relu(self.conv_0(embedded))
        conved_1 = F.relu(self.conv_1(embedded))
        conved_2 = F.relu(self.conv_2(embedded))
        conved_3 = F.relu(self.conv_3(embedded))

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        pooled_3 = F.max_pool1d(conved_3, conved_3.shape[2]).squeeze(2)

        #pooled_n = [batch size, n_fibatlters]

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2, pooled_3), dim = 1).cuda())

        #cat = [batch size, n_filters * len(filter_sizes)]

        result =  self.fc(cat)

        #print(result.shape)

        return result

In [7]:
# Load phobert and cnn

import torch
phoBert = torch.load(r'/content/drive/MyDrive/Google Colab TMP/Bản sao của phobert_cnn_model_part1_task2a_2.pt')
cnn = torch.load(r'/content/drive/MyDrive/Google Colab TMP/Bản sao của phobert_cnn_model_part2_task2a_2.pt')
phoBert.eval()
cnn.eval()

CNN(
  (fc_input): Linear(in_features=768, out_features=768, bias=True)
  (conv_0): Conv1d(768, 32, kernel_size=(1,), stride=(1,))
  (conv_1): Conv1d(768, 32, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  (conv_3): Conv1d(768, 32, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=128, out_features=3, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [11]:
import re
def remove_html(txt):
    return re.sub(r"http\S+", "", txt)

In [31]:
"""
    Start section: Chuyển câu văn về kiểu gõ telex khi không bật Unikey
    Ví dụ: thủy = thuyr, tượng = tuwowngj
"""
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)


def vn_word_to_telex_type(word):
    dau_cau = 0
    new_word = ''
    for char in word:
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            new_word += char
            continue
        if y != 0:
            dau_cau = y
        new_word += bang_nguyen_am[x][-1]
    new_word += bang_ky_tu_dau[dau_cau]
    return new_word


def vn_sentence_to_telex_type(sentence):
    """
    Chuyển câu tiếng việt có dấu về kiểu gõ telex.
    :param sentence:
    :return:
    """
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = vn_word_to_telex_type(word)
    return ' '.join(words)


"""
    End section: Chuyển câu văn về kiểu gõ telex khi không bật Unikey
"""

'\n    End section: Chuyển câu văn về kiểu gõ telex khi không bật Unikey\n'

In [38]:
#Testing
vn_word_to_telex_type('gà')

'gaf'

In [37]:
# -*- coding: utf-8 -*-

import regex as re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)


In [39]:

"""
    Start section: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
    Xem tại đây:

"""


def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


"""
    End section: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
    Xem tại đây: https://vi.wikipedia.org/wiki/Quy_tắc_đặt_dấu_thanh_trong_chữ_quốc_ngữ
"""
if __name__ == '__main__':
    print(chuan_hoa_dau_cau_tieng_viet('anh hoà, đang làm.. gì'))

anh hòa, đang làm.. gì


In [13]:
# lấy dữ liệu cho teencode
import pandas as pd
teencode_df = pd.read_csv('/content/drive/MyDrive/Google Colab TMP/teencode.txt',names=['teencode','map'],sep='\t',)
teencode_list = teencode_df['teencode'].to_list()
map_list = teencode_df['map'].to_list()

In [16]:
# Nltk for connect sent
import nltk
nltk.download('perluniprops')

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.


True

In [18]:
!pip install sacremoses -qq

In [19]:
test = ['Story', ':', 'I', 'wish', 'my', 'dog', "'s", 'hair', 'was', 'fluffier', ',', 'and', 'he', 'ate', 'better']

#from nltk.tokenize.moses import MosesDetokenizer
from sacremoses import MosesTokenizer, MosesDetokenizer
detokens = MosesDetokenizer().detokenize(test, return_str=True)
detokens

"Story: I wish my dog's hair was fluffier, and he ate better"

In [20]:
# For chec a word is an eng word
!apt install enchant
!pip3 install pyenchant

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.7-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.7-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,316 kB of archives.
After this operation, 5,474 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libtext-iconv-perl amd64 1.7-7 [13.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libaspell15 amd64 0.60.8-1ubuntu0.1 [328 kB]
Get:3 http://archive.ubu

In [21]:
import enchant
eng = enchant.Dict("en_US")

In [22]:
sent = 'hello, cute vcllll tộc ttrưởng đi đâu chơi đấyyyyyy :)))))))) ??????. Đc cc, được luôn. Hahahahahahaaaaaaaaaaaaaaaa ?'

def remove_dub_char(sentence):
  sentence = str(sentence)
  words = []
  for word in sentence.strip().split():
    if word in teencode_list:
      words.append(word)
      continue
    if eng.check(str(word)):
      words.append(word)
      continue
    words.append(re.sub(r'([A-Z])\1+', lambda m: m.group(1), word, flags = re.IGNORECASE))
  return ' '.join(words)

def remove_dub_spec_char(sentence):
    sentence = str(sentence)
    words = []
    for word in sentence.strip().split():
        if word in teencode_list:
            words.append(word)
            continue
        if eng.check(str(word)):
            words.append(word)
            continue
        cleaned_word = re.sub(r'([^\w\s])\1+', r'\1', word)
        words.append(cleaned_word)
    return ' '.join(words)

  # #Tokenize
  # tokens_list = word_tokenize(sentence)
  # for idx, word in enumerate(tokens_list):
  #   if word in teencode_list:
  #     continue
  #   if eng.check(str(word)):
  #     continue
  #   worded = re.sub(r'([A-Z])\1+', lambda m: m.group(1), word, flags = re.IGNORECASE)
  #   if worded != word:
  #     noneed_count += 1
  #   tokens_list[idx] = re.sub(r'([A-Z])\1+', lambda m: m.group(1), word, flags = re.IGNORECASE)

  # tokens_list = word_tokenize()
  # nestList_tokens = rdrsegmenter.tokenize(sentence)
  # for tokens_idx, text_tokens in enumerate(nestList_tokens):
  #   lenn += len(text_tokens)
  #   for idx, word in enumerate(text_tokens):
  #     #Neu tu co trong danh sach teencode thi continue
  #     if word in teencode_list:
  #       continue
  #     if eng.check(str(word)):
  #       continue
  #     text_tokens[idx] = re.sub(r'([A-Z])\1+', lambda m: m.group(1), word, flags = re.IGNORECASE)
  #   nestList_tokens[tokens_idx] = text_tokens

  # MAKE LIST FLAT
  # flat_list = [item for sublist in tokens_list for item in sublist]

  # detokens = MosesDetokenizer().detokenize(tokens_list, return_str=True)

  # return detokens

remove_dub_char(sent)

'helo, cute vcl tộc trưởng đi đâu chơi đấy :)))))))) ??????. Đc c, được luôn. Hahahahahaha ?'

In [25]:
def text_preprocess(document):
    global link_count
    global unicode_count
    global dau_count
    global lower_count
    global noneed_count
    global space_count

    document = str(document)

    # đưa về lower
    ducument_before_lower = document
    document = document.lower()
    if ducument_before_lower != document:
      # print("Cau chua xu ly:", ducument_before_lower)
      # print("Cau da xu ly:", document)
      lower_count += 1
    # del document1

    # xóa khoảng trắng thừa
    ducument_before_space = document
    document = re.sub(r'\s+', ' ', document).strip()
    if ducument_before_space != document:
      # print("Cau chua xu ly:", ducument_before_space)
      # print("Cau da xu ly:", document)
      space_count += 1
    # del document1

    # xóa html code
    ducument_before_html = document
    document = remove_html(document)
    if ducument_before_html != document:
      # print("Cau chua xu ly:", ducument_before_html)
      # print("Cau da xu ly:", document)
      link_count += 1
    # del document1

    # chuẩn hóa unicode
    ducument_before_unicode = document
    document = convert_unicode(document)
    if ducument_before_unicode != document:
      unicode_count += 1
    # del document1

    # xóa các ký tự không cần thiết
    ducument_before_redundant = document
    document = remove_dub_char(document)
    if ducument_before_redundant != document:
      # print("Cau chua xu ly:", ducument_before_redundant)
      # print("Cau da xu ly:", document)
      noneed_count += 1
    # del document1


    # chuẩn hóa cách gõ dấu tiếng Việt
    ducument_before_dau = document
    document = chuan_hoa_dau_cau_tieng_viet(document)
    if ducument_before_dau != document:
      print("Cau chua xu ly:", ducument_before_dau)
      print("Cau da xu ly:", document)
      dau_count += 1
    # del document1

    return document

In [40]:
text_preprocess('hiếu THUẬN')

'hiếu thuận'

In [42]:
!pip install spacymoji -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [43]:
!python -m spacy download en_core_web_sm

2023-07-05 03:56:03.745102: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [50]:
text = '       tiên tri nói         quằn qùeeeee hay vcllll :))       ) '
phase1_text = text_preprocess(text)

Cau chua xu ly: tiên tri nói quằn qùe hay vcl :)) )
Cau da xu ly: tiên tri nói quằn què hay vcl :)) )


In [51]:
phase1_text

'tiên tri nói quằn què hay vcl :)) )'

In [52]:
# Word segmenter
!pip3 install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=a6cab630113b3abe3d53c499f89fe10516ee5195b58cb80f93c4a73e186bacea
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
--2023-07-05 04:02:52--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   169MB/s    in 0.2s    

2023-07-05 04:02:53 (169 MB/s) - ‘VnCoreNLP

In [53]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

text = "hello, hòa thuận"

word_segmented_text = rdrsegmenter.tokenize(text)
print(type(word_segmented_text))
print(word_segmented_text)

<class 'list'>
[['hello', ',', 'hoà_thuận']]


In [54]:
def searchTeencode(word):
  try:
    global teencode_count
    index = teencode_list.index(word)
    map_word = map_list[index]
    teencode_count += 1
    return map_word
  except:
    pass

In [55]:
# stopword = ['bị', 'bởi', 'cả', 'các', 'cái', 'cần', 'càng', 'chỉ', 'chiếc', 'cho', 'chứ', 'chưa', 'chuyện', 'có', 'có_thể', 'cứ',
#             'của', 'cùng', 'cũng', 'đã', 'đang', 'đây', 'để', 'đến_nỗi', 'đều', 'điều', 'do', 'đó', 'được', 'dưới', 'gì', 'khi',
#             'không', 'là', 'lại', 'lên', 'lúc', 'mà', 'mỗi', 'một_cách', 'này', 'nên', 'nếu', 'ngay', 'nhiều', 'như', 'nhưng',
#             'những', 'nơi', 'nữa', 'phải', 'qua', 'ra', 'rằng', 'rất', 'rồi', 'sau', 'sẽ', 'so', 'sự', 'tại', 'theo', 'thì', 'trên',
#             'trước', 'từ', 'từng', 'và', 'vẫn', 'vào', 'vậy', 'vì', 'việc', 'với', 'vừa']

STOPWORDS = '/content/drive/MyDrive/Google Colab TMP/vietnamese-stopwords-dash.txt'

# features extraction
with open(STOPWORDS, "r") as ins:
    stopword = []
    for line in ins:
        stopword.append(line.strip('\n'))

def remove_stopwords(line):
    global stopword_count
    words = []
    for word in line:
        if word not in stopword:
            words.append(word)
        if word in stopword:
            stopword_count += 1
    return words

# with open(STOPWORDS, "r") as ins:
#     stop_words = []
#     for line in ins:
#         stop_words.append(line.strip('\n'))

In [56]:
#DE STOPWORD_TEENCODE BY VNCORENLP
stopword_count = 0
teencode_count =0
def stopWords_Teencode(sentence):
  lenn = 0
  sentence = str(sentence)
  #Tokenize
  nestList_tokens = rdrsegmenter.tokenize(sentence)
  for tokens_idx, text_tokens in enumerate(nestList_tokens):
    #Teencode
    lenn += len(text_tokens)
    for idx, word in enumerate(text_tokens):
      deteencoded = searchTeencode(word)
      if(deteencoded != None):
        text_tokens[idx] = deteencoded
    nestList_tokens[tokens_idx] = text_tokens

  # deteencode_sentence = (" ").join(nestList_tokens)

  #Stopwords
  flat_list = [item for sublist in nestList_tokens for item in sublist]

  List_without_sw = remove_stopwords(flat_list)

  #Detokenize
  detokens = MosesDetokenizer().detokenize(List_without_sw, return_str=True)

  return detokens

In [215]:
sent = 'đm điên thật đéo biết phải làm gì luôn á'

In [216]:
stopWords_Teencode(sent)

'địt mẹ điên đéo'

In [217]:
sent_input = stopWords_Teencode(sent)

In [218]:
sent_input

'địt mẹ điên đéo'

In [219]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [220]:
input_ids = []
attention_masks = []
encoded_dict = tokenizer.encode_plus(sent_input,
                                             add_special_tokens=True,
                                             max_length=50,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
input_ids.append(encoded_dict['input_ids'])

attention_masks.append(encoded_dict['attention_mask'])

In [221]:
input_ids = torch.cat(input_ids,dim=0).cuda()
attention_masks = torch.cat(attention_masks,dim=0).cuda()

In [222]:
embedded = phoBert(input_ids,attention_masks)[0]

In [223]:
predictions = cnn(embedded)

In [224]:
 predictions = predictions.detach().cpu().numpy()

In [225]:
predictions

array([[-5.383103 ,  5.0419154, -3.3497438]], dtype=float32)

In [226]:
import numpy as np
prediction = np.argmax(predictions)
prediction

1

In [227]:
if prediction == 0:
  label = "clean"
elif prediction == 1:
  label = 'offensive'
else:
  label = "hate"

In [214]:
print("Cau ban nhap la:" + sent + "\n")
print("Ket qua la:" + label)

Cau ban nhap la:thằng đó đẹp trai vl, đéo thể tin được nhưng mà còn ngu nữa, haha. Con cặc, đjt mẹ, thg đàn bà bê đê chúa

Ket qua la:offensive


In [239]:
import numpy as np
def predict(sent_input):
  sent_input = stopWords_Teencode(sent_input)
  input_ids = []
  attention_masks = []
  encoded_dict = tokenizer.encode_plus(sent_input,
                                             add_special_tokens=True,
                                             max_length=50,
                                             padding='longest',
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
  input_ids.append(encoded_dict['input_ids'])

  attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids,dim=0).cuda()
  attention_masks = torch.cat(attention_masks,dim=0).cuda()

  embedded = phoBert(input_ids,attention_masks)[0]
  predictions = cnn(embedded)
  predictions = predictions.detach().cpu().numpy()
  prediction = np.argmax(predictions)
  if prediction == 0:
    label = "clean"
  elif prediction == 1:
    label = 'offensive'
  else:
    label = "hate"

  print("Cau ban nhap la:" + sent + "\n")
  print("Ket qua la:" + label)

In [246]:
sent = "Xe Việt Nam vẫn quá đắt và đường xá thì quá kém"
predict(sent)

Cau ban nhap la:Xe Việt Nam vẫn quá đắt và đường xá thì quá kém

Ket qua la:clean
